In [3]:
import requests
import gspread
from google.oauth2.service_account import Credentials
from datetime import datetime, timezone

In [4]:
def get_weather_data(city, api_key):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return {
            "Timestamp": datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S'), 
            "City": city,
            "Temperature (°C)": data["main"]["temp"],
            "Humidity (%)": data["main"]["humidity"],
            "Description": data["weather"][0]["description"],
            "Time": datetime.fromtimestamp(data["dt"], tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S'),
            "Sunrise": datetime.fromtimestamp(data["sys"]["sunrise"], tz=timezone.utc).strftime('%H:%M:%S'),
            "Sunset": datetime.fromtimestamp(data["sys"]["sunset"], tz=timezone.utc).strftime('%H:%M:%S'),
            "Wind Speed (m/s)": data["wind"]["speed"],
            "Weather Icon": data["weather"][0]["icon"]
        }
    else:
        print(f"Error fetching data for {city}")
        return None

In [5]:
def connect_to_gsheet(json_path, sheet_name):
    scope = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
    creds = Credentials.from_service_account_file(json_path, scopes=scope)
    client = gspread.authorize(creds)
    sheet = client.open(sheet_name).sheet1

    # Ensure headers exist
    if not sheet.row_values(1):
        sheet.append_row([
            "Timestamp", "City", "Temperature (°C)", "Humidity (%)", "Description", 
            "Time", "Wind Speed (m/s)", "Sunrise", "Sunset", "Weather Icon"
        ])
    return sheet

In [6]:
api_key = "8e1981068d7b5bb0e6d60d0792007914"
json_path = r"C:\data\data-with-stephen-457108-a8e720844e4b.json"
cities = ["New York", "London", "Lagos", "Delhi", "Tokyo"]  # Add your cities
sheet = connect_to_gsheet(json_path, "WeatherData")

In [ ]:
# Get all the cities already in the sheet (from column 2, which is "City")
existing_cities = sheet.col_values(2)[1:]  # Skip header

# Ask user for a city
city = input("Enter a city name: ")

# Check if city is already in the sheet
if city in existing_cities:
    print(f"{city} already exists in the sheet. Skipping entry.")
else:
    # Collect weather data
    data = get_weather_data(city, api_key)
    if data:
        # Append new weather data to the sheet
        row = [
            data["Timestamp"],
            data["City"],
            data["Temperature (°C)"],
            data["Humidity (%)"],
            data["Description"],
            data["Time"],
            data["Wind Speed (m/s)"],
            data["Sunrise"],
            data["Sunset"],
            data["Weather Icon"]
        ]
        sheet.append_row(row)
        print(f"📤 Weather for {city} added to Google Sheets!")
    